1- Preprocessing :
define characters accepted and tashkeel accepted and remove from the training set any tashkeel and unwanted characters (eg. brackets, numbers... etc)

In [23]:
import re
import pyarabic.araby as araby

In [24]:
tashkeel = ["ْ", "ّ", "ٌ", "ٍ", "ِ", "ً", "َ", "ُ"]
arabicCharacters = "ىعظحرسيشضقثلصطكآماإهزءأفؤغجئدةخوبذتن"
punctuations = [".", "،", ":", "؛", "-", "؟"]
validCharacters= tashkeel + list(arabicCharacters) + punctuations
charcters_without_tashkeel = list(arabicCharacters) + punctuations

In [25]:
#reading the training dataset
f = open(r"train.txt", "r",encoding="utf-8").read()
print(type(f))

<class 'str'>


In [26]:
print(len(f))
#regex to keep arabic letters only and remove any other character (eg. brackets, numbers ...etc)
characters_regex =r'[\s\.\u0600-\u06ff\u0750-\u077f\ufb50-\ufbc1\ufbd3-\ufd3f\ufd50-\ufd8f\ufd50-\ufd8f\ufe70-\ufefc\uFDF0-\uFDFD]+'
processedData = re.findall(characters_regex,f)
processedData = " ".join(processedData)
processedData = re.sub(r"\s+"," " ,processedData) #substitute many spaces with one space only
print(len(processedData))

18515271
17997403


In [27]:
w = open('processed.txt', 'w')
w.write(processedData)
w.close()

In [29]:
#remove diacritics
without_diacritics= araby.strip_diacritics(processedData)
print(len(without_diacritics))
w2 = open('withoutDiacritics.txt', 'w')
w2.write(without_diacritics)
w2.close()

10711561
